# Database for the Recipies 

Import all the Important Modules

In [1]:
import json
import os
import sqlite3

In [2]:
#connection to DB
con = sqlite3.connect("RecipeDB.db")

#create cursor
cursor = con.cursor()

![alt text](Database.png "Datenbank")


Creating the Database


In [3]:


#Recipe Table
sql = """
CREATE TABLE Recipe(
recID TEXT PRIMARY KEY NOT NULL,
recName TEXT,
recHeadline TEXT,
recDisc BLOB,
recDifficulty INT,
recPrepTime TEXT, 
recTotalTime TEXT,
recServingSize INT,
recLink TEXT,
recRating REAL,
recRatingCount INT,
recSteps INT
);
"""

cursor.execute(sql)

#IngredientFamily Table

sql = """
CREATE TABLE IngFamily(
familyID TEXT PRIMARY KEY NOT NULL,
familyName TEXT
);

"""

cursor.execute(sql)

#Ingredients TAble
sql = """
CREATE TABLE Ingredients(
ingID TEXT PRIMARY KEY NOT NULL,
ingName TEXT NOT NULL,
familyID TEXT NOT NULL,
FOREIGN KEY(familyID) REFERENCES IngFamily(familyID)
);

"""

cursor.execute(sql)


#Ingredients in Recipe Table
sql = """
CREATE TABLE IngInRec(
ingID TEXT NOT NULL,
recID TEXT NOT NULL,
amount REAL,
unit TEXT,
PRIMARY KEY(ingID, recID),
FOREIGN KEY (ingID) REFERENCES Ingredients(ingID),
FOREIGN KEY (recID) REFERENCES Recipe(recID)
);

"""
cursor.execute(sql)

#Nutartion Table plus NutrInRec
sql = """
CREATE TABLE Nutrations(
nType TEXT PRIMARY KEY NOT NULL,
nName TEXT NOT NULL
);
"""

cursor.execute(sql)

sql = """
CREATE TABLE NutrInRec(
nType TEXT NOT NULL,
recID TEXT NOT NULL,
amount INT,
unit TEXT,
PRIMARY KEY(nType, recID),
FOREIGN KEY (nType) REFERENCES Nutration(nType),
FOREIGN KEY (recID) REFERENCES Recipe(recID)
);
"""

cursor.execute(sql)


#Tags and TagsInRecipe

sql = """
CREATE TABLE Tags(
tagID TEXT PRIMARY KEY NOT NULL,
tagName TEXT,
tagType TEXT
);

"""

cursor.execute(sql)

sql = """
CREATE TABLE TagsInRec(
tagID TEXT NOT NULL,
recID TEXT NOT NULL,
PRIMARY KEY (tagID, recID),
FOREIGN KEY (tagID) REFERENCES Tags(tagID),
FOREIGN KEY (recID) REFERENCES Recipe(recID)
);

"""

cursor.execute(sql)

#Allergens and it´s conTable
sql = """
CREATE TABLE Allergens(
alleID TEXT PRIMARY KEY NOT NULL,
alleType TEXT,
alleName TEXT,
alleSlug TEXT,
alleDisc BLOP,
allTraces BOOLEAN NOT NULL CHECK(allTraces IN (0,1)),
triggersTracesOf BOOLEAN NOT NULL CHECK(triggersTracesOf IN (0,1))

);

"""
cursor.execute(sql)

sql = """
CREATE TABLE AlleInRec(
alleID TEXT NOT NULL,
recID TEXT NOT NULL,
PRIMARY KEY (alleID,recID),
FOREIGN KEY (alleID) REFERENCES Allergens(alleID),
FOREIGN KEY (recID) REFERENCES Recipe(recID)

);

"""
cursor.execute(sql)

#steps table
sql = """
CREATE TABLE StepsInRec(
recID TEXT NOT NULL,
stepIndex INT NOT NULL,
stepDisc BLOP,
PRIMARY KEY (recID, stepIndex),
FOREIGN KEY (recID) REFERENCES Recipe(recID)
);
"""
cursor.execute(sql)

Inserting the Date from the JSON files

In [3]:

test = cursor.execute(""" SELECT * FROM Recipe WHERE recID = '554b1a294dab71a6058b4570' """).fetchall()
print(len(test))

0


In [4]:
path = os.getcwd()+"/rezepte"
#sql to check if there is * already in DB
recipe_check = """ SELECT * FROM Recipe WHERE recID = ? ;"""
ingFamCheck = """SELECT * FROM IngFamily WHERE familyID = ? ;"""
ing_check = """ SELECT * FROM Ingredients WHERE ingID = ? ;"""
Nutr_check = """SELECT * FROM Nutrations WHERE nType = ? ;"""
allg_check = """ SELECT * FROM Allergens WHERE alleID = ? ; """
tag_check = """SELECT * FROM Tags WHERE tagID = ? ;"""
#somehow there errors in the json so some ingredients,tags,allergens or Nutr  are doubled in the yields so we need another check 
iir_check = """SELECT * FROM IngInRec WHERE recID = ? AND ingID = ? ;"""
air_check = """SELECT * FROM AlleInRec WHERE recID = ? AND alleID = ? ;"""
nir_check = """SELECT * FROM NutrInRec WHERE recID = ? AND nType = ? ;"""
tir_check = """SELECT * FROM TagsInRec WHERE recID = ? AND tagID = ? ;"""

#insert sql
recipe_sql =""" INSERT INTO Recipe VALUES (?,?,?,?,?,?,?,?,?,?,?,?);""" 
ingFam_sql = """INSERT INTO IngFamily VALUES (?,?);"""
ing_sql = """INSERT INTO Ingredients VALUES (?,?,?);""" 
alle_sql = """INSERT INTO Allergens VALUES (?,?,?,?,?,?,?);""" 
nutr_sql = """INSERT INTO Nutrations VALUES (?,?);"""
tags_sql = """INSERT INTO Tags VALUES (?,?,?);""" 
steps_sql = """INSERT INTO StepsInRec VALUES (?,?,?);""" 
air_sql = """INSERT INTO AlleInRec VALUES (?,?);"""
iir_sql = """INSERT INTO IngInRec VALUES (?,?,?,?);"""
nir_sql = """INSERT INTO NutrInRec VALUES (?,?,?,?);"""
tir_sql = """INSERT INTO TagsInRec VALUES (?,?);""" 



with_errors = []

without = 0
for f in os.listdir(path):
    with open(os.path.join(path,f), encoding = 'utf8') as file:
        data = json.load(file)
        
        #check if recipe has ingredients
        if not data['ingredients']:
            without += 1
            continue
            
        #check if recipeID is already in DB
        if len(cursor.execute(recipe_check,(data['id'],)).fetchall()):
            without -= -1
            continue
        try:
            #add recipe
            temp = (data['id'],data['name'],data['headline'],data['description'],data['difficulty'],data['prepTime'],data['totalTime'],data['servingSize'],data['cardLink'],data['averageRating'], data['ratingsCount'],len(data['steps']))
            cursor.execute(recipe_sql, temp)
            #add ingredients and all its tables

            for ing in data['ingredients']:
                if len(cursor.execute(ing_check,(ing['id'],)).fetchall()):
                    continue
                # TODO somehow there is a NoneType in some json find a solution bruuuuuuuuuuuuh
                if not len(cursor.execute(ingFamCheck,(ing['family']['id'],)).fetchall()):
                    fam = ing['family']
                    temp = (fam['id'],fam['name'])
                    cursor.execute(ingFam_sql,temp)

                temp = (ing['id'],ing['name'],ing['family']['id'])
                cursor.execute(ing_sql,temp)

            #make a check cuz the ingriedients per recipe arent unique bruuuuuuuuuuuuuuuuuuh
            y = data['yields'][0]
            div = int(y['yields'])
            for ing in y['ingredients']:
                if len(cursor.execute(iir_check,(data['id'],ing['id'])).fetchall()):
                    continue
                #print(ing)
                am = int(ing['amount']) if ing['amount'] else 0
                am /= div
                temp = (ing['id'],data['id'],am,ing['unit'])
                cursor.execute(iir_sql,temp)

            #nutr
            for n in data['nutrition']:
                if len(cursor.execute(nir_check,(data['id'],n['type'])).fetchall()):
                    continue
                if not len(cursor.execute(Nutr_check,(n['type'],)).fetchall()):
                    cursor.execute(nutr_sql,(n['type'],n['name']))
                cursor.execute(nir_sql,(n['type'],data['id'],n['amount'],n['unit']))

            #Tags
            for t in data['tags']:
                if len(cursor.execute(tir_check,(data['id'],t['id'])).fetchall()):
                    continue
                if not len(cursor.execute(tag_check,(t['id'],)).fetchall()):
                    cursor.execute(tags_sql,(t['id'],t['name'],t['type']))
                cursor.execute(tir_sql,(t['id'],data['id']))

            #Allergens
            for a in data['allergens']:
                if len(cursor.execute(air_check,(data['id'],a['id'])).fetchall()):
                    continue
                if not len(cursor.execute(allg_check,(a['id'],)).fetchall()):
                    cursor.execute(alle_sql,(a['id'],a['type'],a['name'],a['slug'],a['description'],a['tracesOf'],a['triggersTracesOf']))
                cursor.execute(air_sql,(a['id'],data['id']))


            #Steps
            for s in data['steps']:
                cursor.execute(steps_sql,(data['id'],s['index'],s['instructions']))
        except:
            without += 1
            with_errors_append(data['id'])
        con.commit()
            
cursor.close()
        
        
        



TypeError: 'NoneType' object is not subscriptable